#### Paphos Weather

This page shows various metrics for the weather in Paphos, Cyprus.

**For the Hours 13:00 to 16:00, the hottest part of the day**

Weather data is purchased from Openweather Marketplace [here](https://home.openweathermap.org/marketplace/my_orders)

Download this weather data from here https://drive.google.com/file/d/14GgJu7YfZulZU9_xpS9GVcykdb1nEaUV/view?usp=sharing




In [112]:

from datetime import date
from datetime import datetime
import time
import pandas as pd

def toDate(epoch_time):
    return datetime.fromtimestamp(epoch_time)

def calculate_heat_index(temperature, relative_humidity):
    heat_index = int((-8.784695 +
                  1.61139411 * temperature +
                  (2.338549 * relative_humidity) +
                  (-0.14611605 * temperature * relative_humidity) +
                  (-0.01230809 * (temperature ** 2)) +
                  (-0.01642482 * (relative_humidity ** 2)) +
                  (0.00221173 * (temperature ** 2) * relative_humidity) +
                  (0.00072546 * temperature * (relative_humidity ** 2)) +
                  (-0.000003582 * (temperature ** 2) * (relative_humidity ** 2))))
    return heat_index

In [113]:
import pandas as pd

df=pd.read_json('/Users/walkerrowe/Documents/hypatia/paphosWeather2023.json')

df['dt'].map(toDate)

0       2018-07-01 03:00:00
1       2018-07-01 04:00:00
2       2018-07-01 05:00:00
3       2018-07-01 06:00:00
4       2018-07-01 07:00:00
                ...        
43843   2023-07-01 22:00:00
43844   2023-07-01 23:00:00
43845   2023-07-02 00:00:00
43846   2023-07-02 01:00:00
43847   2023-07-02 02:00:00
Name: dt, Length: 43848, dtype: datetime64[ns]

In [114]:
df['month']=df['dt'].map(lambda l: toDate(l).strftime("%m"))
df['day']=df['dt'].map(lambda l: toDate(l).strftime("%d"))
df['year']=df['dt'].map(lambda l: toDate(l).strftime("%y"))
df['hour']=df['dt'].map(lambda l: toDate(l).strftime("%H"))
 

In [115]:
df['temp']=df["main"].map(lambda x: int(x["temp"]))
df['temp_min']=df["main"].map(lambda x: int(x["temp_min"]) )
df['temp_max']=df["main"].map(lambda x: int(x["temp_max"]) )
df['humidity']=df["main"].map(lambda x: int(x["humidity"]))



In [116]:
df['heatIndex'] = df.apply(lambda row: calculate_heat_index(row['temp'], row['humidity']), axis=1)
df.set_index("dt_iso",inplace=True)

In [117]:
summer1=df.loc[df['month'].isin(['06','07','08'])]
summer=summer1.loc[summer1['hour'].isin(['13','14','15','16'])]

In [118]:
hot=summer[['year','month','day', 'hour' ,'temp','temp_min','temp_max','hour', 'humidity', 'heatIndex']]

We want to caculate the average temperature by date usign the mean() function.  But the problem is that the date is an object inside the **main** object.  See below how to fix that.

In [119]:
hot.sort_values(by='year',ascending=False).head(15)

,year,month,day,hour,temp,temp_min,temp_max,hour,humidity,heatIndex
dt_iso,,,,,,,,,,
2023-07-01 13:00:00 +0000 UTC,23,07,01,16,26,24,32,16,61,26
2023-06-12 11:00:00 +0000 UTC,23,06,12,14,23,21,26,14,61,24
2023-06-09 10:00:00 +0000 UTC,23,06,09,13,25,22,28,13,57,25
2023-06-09 11:00:00 +0000 UTC,23,06,09,14,25,22,28,14,51,25
2023-06-09 12:00:00 +0000 UTC,23,06,09,15,25,22,27,15,61,25
2023-06-09 13:00:00 +0000 UTC,23,06,09,16,25,22,27,16,51,25
2023-06-10 10:00:00 +0000 UTC,23,06,10,13,23,22,26,13,69,24
2023-06-10 11:00:00 +0000 UTC,23,06,10,14,23,22,26,14,73,23
2023-06-10 12:00:00 +0000 UTC,23,06,10,15,23,21,26,15,74,23


In [120]:
years=hot.groupby('year')

In [121]:
average_colums=years[['temp','temp_min','temp_max','humidity','heatIndex']]

In [122]:
average_colums.mean().round(0).astype(int).sort_values(by='temp',ascending=False)

,temp,temp_min,temp_max,humidity,heatIndex
year,,,,,
18,28,26,30,68,30
19,28,27,30,67,30
20,28,28,31,66,31
21,27,25,32,60,29
22,27,25,31,61,28
23,24,22,27,63,25


In [123]:
hours=summer.groupby(['month'])

In [124]:
month=hours[['temp','temp_min','temp_max','humidity','heatIndex']]
month.mean().round(0).astype(int).sort_values(by='temp',ascending=False)

,temp,temp_min,temp_max,humidity,heatIndex
month,,,,,
08,29,27,32,64,31
07,28,27,32,64,30
06,25,24,28,64,26
